# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [48]:
# import libraries
import pandas as pd
from sqlalchemy import create_engine
import re
from pprint import pprint

import nltk
nltk.download(['punkt', 'stopwords', 'wordnet'])
import numpy as np
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC, LinearSVC
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.preprocessing import StandardScaler

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\yma80\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\yma80\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\yma80\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [49]:
# load data from database
def load_data(db_filepath='DisasterResponse.db', sample_frac=1.0):
    """Load the data from database.
    Args:
        db_filepath: str
            Filepath to the SQL database.
        sample_frac: int
            Apply random subsampling and select only frac of entries.
    Returns:
        X, Y: np.ndarray or pandas.DataFrame
            Matrices of predictors (messages) and responses.
    """
    
    engine = create_engine('sqlite:///'+db_filepath)
    df = pd.read_sql_table('merged', engine).sample(frac=sample_frac, random_state=42)
    X = df['message']
#     Y = df[['related', 'request', 'offer', 'aid_related', 'infrastructure_related', 'weather_related']]
#     Y = df[ [c for c in df.columns if 'related' in c] ]
    Y = df[ [c for c in df.columns if c not in ('id', 'message', 'original', 'genre', 'child_alone')] ]
#     Y = Y.replace({'genre':{'direct':0, 'news':1, 'social':2}})

    category_names = Y.columns

    return X, Y, category_names

def display_results(y_test, y_pred):
    labels = np.unique(y_pred)
    confusion_mat = confusion_matrix(y_test, y_pred, labels=labels)
    accuracy = (y_pred == y_test).mean()

    print("Labels:", labels)
    print("Confusion Matrix:\n", confusion_mat)
    print("Accuracy:", accuracy)
    
engine = create_engine('sqlite:///DisasterResponse.db')
df = pd.read_sql_table('merged', engine).sample(frac=1.0, random_state=42)
X = df['message']
Y = df[ [c for c in df.columns if c not in ('id', 'message', 'original')] ]
df.describe()

df.columns

Index(['id', 'message', 'original', 'genre', 'related', 'request', 'offer',
       'aid_related', 'medical_help', 'medical_products', 'search_and_rescue',
       'security', 'military', 'water', 'food', 'shelter', 'clothing', 'money',
       'missing_people', 'refugees', 'death', 'other_aid',
       'infrastructure_related', 'transport', 'buildings', 'electricity',
       'tools', 'hospitals', 'shops', 'aid_centers', 'other_infrastructure',
       'weather_related', 'floods', 'storm', 'fire', 'earthquake', 'cold',
       'other_weather', 'direct_report'],
      dtype='object')

In [3]:
# Investigate the hierarchy of responses
df['req_or_off'] = df.request | df.offer
df['any_related'] = df.aid_related | df.infrastructure_related | df.weather_related

df1 = df.query("related == 1")

df1.query("req_or_off==0")[['req_or_off', 'direct_report']+[c for c in df.columns if 'related' in c]].describe()

df1.query("offer==1").message

11096    I have many towels ( large , as well as hand t...
11293    When and Where should hot meals be delivered t...
11335    We did a large donation drive in our community...
75             I am a driver, a mechanic ,. I want to help
23986    Following the attack on Ngouboua village by ar...
10646                    Clothes sorting . Giving out food
10854    I can bring batteries , toiletries , non-peris...
11202    Looking to donate new ( several-year shelf lif...
25301    The returnees were provided with UNHCR's stand...
18098    - UNFPA, WHO and UNICEF are building over 100 ...
10992    I 'm a nutritionist and would be also happy to...
10607    i 'm also good with food prep if there 's a ne...
21684    UNICEF has already sent supplies to affected a...
11187    Will provide & serve nonperishable food . Also...
25201    MSF teams are active with one Zodiac (inflatab...
25291    World Bank Country Director Xian Zhu told a pr...
18008    This assistance, valued at $200.7 million, was.

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [50]:
X, Y, cat_names = load_data(sample_frac=0.1)
X_train_txt, X_test_txt, Y_train, Y_test = train_test_split(X, Y, random_state=42, test_size=0.25)

In [5]:
class StartingVerbExtractor(BaseEstimator, TransformerMixin):

    def starting_verb(self, text):
        sentence_list = nltk.sent_tokenize(text)
        for sentence in sentence_list:
            pos_tags = nltk.pos_tag(tokenize(sentence))
            first_word, first_tag = pos_tags[0]
            if first_tag in ['VB', 'VBP'] or first_word == 'RT':
                return True
        return False

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X_tagged = pd.Series(X).apply(self.starting_verb)
        return pd.DataFrame(X_tagged)
    
class DenseTransformer(TransformerMixin):
    """Transforms a sparse matrix to its dense representation."""

    def fit(self, X, y=None, **fit_params):
        return self

    def transform(self, X, y=None, **fit_params):
        return X.todense()

NameError: name 'tokenize' is not defined

In [51]:
class LengthEstimator(BaseEstimator, TransformerMixin):
    """A custom transformer class to estimate the length of a message."""

    def fit(self, X, y=None, **fit_params):
        return self

    def transform(self, X, y=None, **fit_params):
        return pd.DataFrame(pd.Series(X).str.len())

def tokenize(text):
    """Turn a text string into an array of words.
        Args:
            text: str
        Returns:
            A list of lemmatised tokens (words).
    """
    
    text = text.lower()
    text = re.sub(r'[^a-z]', ' ', text)
    
    text = nltk.word_tokenize(text)
    
    text = [w for w in text if w not in stopwords.words('english')]
    
    # Reduce words to their root form
    text = [WordNetLemmatizer().lemmatize(w, pos='v') for w in text]
    
    return text

def build_model(cv=True):
    """Define and train the classification pipeline.

        The pipeline takes as its input a string representing a text message,
        transforms it into TF-IDF vector, and runs a multi-target classification
        algorithm with 36 targets.
        
    Args:
        cv: bool
            If True, will perform parameter search with cross-validation.
                
    """

    # A (sub-)pipeline to vectorize the message and ombine it with a text-length feature
    pl_NLP = FeatureUnion([
                    ('pl_Txt', Pipeline([
                                        ('vectr', CountVectorizer(tokenizer=tokenize, max_features=1500)),
                                        ('tfidf', TfidfTransformer()),
                                ]) ),
                    ('length', LengthEstimator())
                ])

    # Add an SVC classifier
    pl_Final = Pipeline([
                    ('pl_NLP', pl_NLP),
                    ('clsfr', MultiOutputClassifier(SVC(gamma='auto')))
               ])
    
    # Run GridSearch to optimize some of the parameters
    parameters = {'pl_NLP__pl_Txt__vectr__max_df' : [1.0, 0.9],
              'clsfr__estimator__kernel' : ['rbf', 'poly']}
    
    # Run parameter search with cross-validation
    if cv:
        cv = GridSearchCV(pl_Final, param_grid=parameters, cv=3, verbose=4, scoring='f1_weighted')
        return cv
    else:
        return pl_Final

model = build_model(cv=False)

model.fit(X_train_txt, Y_train)

Pipeline(memory=None,
     steps=[('pl_NLP', FeatureUnion(n_jobs=None,
       transformer_list=[('pl_Txt', Pipeline(memory=None,
     steps=[('vectr', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0...ity=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False),
           n_jobs=None))])

In [52]:
def evaluate_model(model, X_test, Y_test, cat_names):
    """Evaluate the trained model and display classification reports.
        Args:
            X_test, Y_test : pd.DataFrame or np.array
                Testing sets of predictors (messages) and labels.
            cat_names: list of str
                Names of the classification categories.
        Returns:
            Y_pred: pd.DataFrame
                Predicted labels.
    """

    Y_pred = pd.DataFrame(model.predict(X_test), columns = cat_names)
    
    for i, cat in enumerate(cat_names):
        print('Classification report for category #{:d} - {}'.format(i, cat))
        print(classification_report(Y_test[cat], Y_pred[cat]))
    
    return Y_pred

Y_pred = evaluate_model(model, X_test_txt, Y_test, cat_names)

Classification report for category #0 - related
              precision    recall  f1-score   support

           0       0.50      0.13      0.21       153
           1       0.78      0.96      0.86       502

   micro avg       0.77      0.77      0.77       655
   macro avg       0.64      0.55      0.54       655
weighted avg       0.72      0.77      0.71       655

Classification report for category #1 - request
              precision    recall  f1-score   support

           0       0.82      1.00      0.90       535
           1       0.00      0.00      0.00       120

   micro avg       0.82      0.82      0.82       655
   macro avg       0.41      0.50      0.45       655
weighted avg       0.67      0.82      0.73       655

Classification report for category #2 - offer
              precision    recall  f1-score   support

           0       0.99      1.00      1.00       651
           1       0.00      0.00      0.00         4

   micro avg       0.99      0.99      0

C:\Users\yma80\AppData\Local\Continuum\anaconda3\envs\py35\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.



In [53]:
engine = create_engine('sqlite:///DisasterResponse.db')
df = pd.read_sql_table('merged', engine).sample(frac=1.0, random_state=42)
classification_labels = df.columns[4:]
print(classification_labels)
print(len(classification_labels))

Index(['related', 'request', 'offer', 'aid_related', 'medical_help',
       'medical_products', 'search_and_rescue', 'security', 'military',
       'water', 'food', 'shelter', 'clothing', 'money', 'missing_people',
       'refugees', 'death', 'other_aid', 'infrastructure_related', 'transport',
       'buildings', 'electricity', 'tools', 'hospitals', 'shops',
       'aid_centers', 'other_infrastructure', 'weather_related', 'floods',
       'storm', 'fire', 'earthquake', 'cold', 'other_weather',
       'direct_report'],
      dtype='object')
35


In [38]:
# Try prediction
for msg in df.query("related == 1").message:
    print(model.predict([msg]).ravel())
    print(msg)

[1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
The overall severity of a pandemic is further influenced by the tendency of pandemics to encircle the globe in at least two, sometimes three, waves, the WHO said.
[1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
My wife is pregnant and suffering, I have a 16 month old son
[1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
i would like to known some numbers ,adresses of contact office, so we can paticipe on the united nation program. we live at damien(tabarre) 
[1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
IF IN REALITY THE GOVERNMENT IS THINKING FOR (OR ABOUT) ALL THOSE PEOPLE. 
[1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
The Maharashtra government has sanctioned Rs 1.5 crore to set up a 66 kv substation neat Bhachau.
[1 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
Over 100 villagers in tsunami 

[1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
On the outskirts of Banda Aceh, the devastated provincial capital, wooden barracks were being constructed over open drains with water pooling underneath.
[1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
Santo 2 and Impasse Beb Blanc. We need tents. The rain is coming down and we are in it othe raine. 
[1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
We are in Plateau Central in Maisade, there are many victims who came here,and what we had is not enough for them, please help us 
[1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
Help us to build some water wells so that we can drink clean water and become healthier."
[1 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
It could also provide pointers for other Asian countries, such as the Philippines, in how to untangle seemingly intractable separatist rebellions over long-standing 

[1 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
why don't you ever past by me ,I don't have any where to sleeps, I'm starving.I see that is only one part that is finding what you're doing isn't wreat,I'm waiting for your answer. 
[1 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
Preparations were underway for a mass cremation in the flood-ravaged holy town of Kedarnath, with rescue workers ordered to collect tonnes of fire wood, amid concerns of an outbreak of disease from rotting bodies, officials said.
[1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
In the area of water and sanitation, testing of water supplies indicate that recontamination of tubewells has not been as widespread as first feared.
[1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
The Samastipur division was very severely affected with heavy damage caused at about 40 places due to breaches in railway tracks, sinkage of embankmen

KeyboardInterrupt: 

## Ploting with Plotly

In [19]:
import plotly
print(int(plotly.__version__[0]) == 4)

True


In [58]:
# Count related and unrelated messages
counts = df[['related', 'request', 'offer', 'weather_related', 'aid_related', 'infrastructure_related']].sum()
counts['unrelated'] = len(df)-counts['related']
counts['related'] -= (counts.request+counts.offer)
print(counts)

# Related messages by different categories

related                   15294
request                    4464
offer                       118
weather_related            7286
aid_related               10841
infrastructure_related     1705
unrelated                  6304
dtype: int64


In [12]:
df[['weather_related', 'aid_related', 'infrastructure_related']].sum(axis=1).value_counts()

0    12177
1     9059
2     4059
3      885
dtype: int64

In [75]:
from plotly.subplots import make_subplots
from plotly.graph_objects import Bar, Sunburst, Pie

# Number of related or unrelated messages
n_related = df.related.value_counts().rename(index={0:'Unrelated', 1:'Related'})

# First figure
fig = make_subplots(rows=1, cols=2, shared_yaxes=True,
                    specs=[[{"type": "pie"}, {"type": "bar"}]])

fig.add_trace(Sunburst(
    labels=["Total", "Related", "Unrelated", "Request", "Offer"],
    parents=["", "Total", "Total", "Related", "Related"],
    values=[0, counts.related, counts.unrelated, counts.request, counts.offer],
    hoverinfo='label+percent parent'
),
              1, 1)

fig.add_trace(Bar(
                    x=["Weather", "Aid", "Infrastructure"],
                    y=counts[['weather_related', 'aid_related', 'infrastructure_related']]
                ), 
        1, 2)

fig.update_layout(showlegend=False,
                  title='Distribution of Messages by Type',
                  yaxis_title='Number of messages',
                  xaxis_title='Related to ...')
fig.show()

# Convert to dictionary to be used on the web page

# pprint(fig.to_dict())


# # Second figure
# fig = make_subplots(rows=1, cols=2, shared_yaxes=True,
#                     specs=[[{"type": "bar"}, {"type": "pie"}]])

# fig.add_trace(Bar(
#     x=["Weather", "Aid", "Infrastructure"],
#     y=counts[['weather_related', 'aid_related', 'infrastructure_related']]
# ),
#               1, 1)

# fig.add_trace(Pie(values=n_related.values, labels=n_related.index,
#                      textinfo='label+percent', hole=0.4),
#               row=1, col=2)

# fig.update_layout(showlegend=True)
# fig.show()